In [1]:
import os

os.environ["KERAS_BACKEND"] = "torch"

In [2]:
import bayesflow as bf
import keras
import numpy as np

In [3]:
def model_parameters():
    recognition = np.random.beta(a=2, b=2)
    guessing = np.random.beta(a=2, b=2)
    
    return dict(recognition=recognition, guessing=guessing)

In [4]:
def model_1ht(recognition, guessing):
    p00 = 1 - guessing
    p01 = guessing
    p10 = (1 - recognition) * (1 - guessing)
    p11 = recognition + (1 - recognition) * guessing
    
    p = np.array([
        [p00, p01],
        [p10, p11],
    ])
    
    return dict(p=p)

In [5]:
def model_2ht(recognition, guessing):
    p00 = recognition + (1 - recognition) * (1 - guessing)
    p01 = (1 - recognition) * guessing
    p10 = (1 - recognition) * (1 - guessing)
    p11 = recognition + (1 - recognition) * guessing
    
    p = np.array([
        [p00, p01],
        [p10, p11],
    ])
    
    return dict(p=p)

In [6]:
def response(p, n_obs=128):    
    response_old = np.random.binomial(n=1, p=p[1, 1], size=(n_obs,))
    response_new = np.random.binomial(n=1, p=p[0, 1], size=(n_obs,))
    
    return dict(response=np.stack([response_old, response_new], axis=-1))

In [7]:
model_1ht = bf.simulators.CompositeLambdaSimulator([model_parameters, model_1ht, response])
model_2ht = bf.simulators.CompositeLambdaSimulator([model_parameters, model_2ht, response])

In [8]:
example_batch = model_1ht.sample((256,), n_obs=64)
example_shapes = keras.tree.map_structure(keras.ops.shape, example_batch)
example_dtypes = keras.tree.map_structure(keras.ops.dtype, example_batch)

print("Shapes:", example_shapes)
print("Dtypes:", example_dtypes)

Shapes: {'recognition': (256, 1), 'guessing': (256, 1), 'p': (256, 2, 2), 'response': (256, 64, 2)}
Dtypes: {'recognition': 'float32', 'guessing': 'float32', 'p': 'float32', 'response': 'int64'}


In [9]:
data_adapter = bf.approximators.ModelComparisonApproximator.build_data_adapter(
    summary_variables=["response"],
)

In [10]:
approximator = bf.approximators.ModelComparisonApproximator(
    num_models=2,
    data_adapter=data_adapter,
    # use super-small networks because this is an easy task
    classifier_network=bf.networks.MLP(depth=2, width=256),
    summary_network=bf.networks.DeepSet(summary_dim=32, depth=1, dropout=None, activation="relu")
)

In [11]:
approximator.compile(optimizer="adam")

In [12]:
approximator.fit(
    batch_size=32,
    data_adapter=data_adapter,
    epochs=10,
    memory_budget="2 GB",
    num_batches=1000,
    simulators=[model_1ht, model_2ht],
    max_queue_size=1000,
)

INFO:bayesflow:Building model comparison simulator from 2 simulators.
INFO:bayesflow:Building dataset from simulator instance of ModelComparisonSimulator.
INFO:bayesflow:Using 16 data loading workers.
INFO:bayesflow:Building on a test batch.


Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 28s 28ms/step - classifier/loss: 0.6453 - loss: 0.6453
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 25s 25ms/step - classifier/loss: 0.5307 - loss: 0.5307
Epoch 3/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 25s 25ms/step - classifier/loss: 0.5370 - loss: 0.5370
Epoch 4/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 25s 25ms/step - classifier/loss: 0.5276 - loss: 0.5276
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 25s 25ms/step - classifier/loss: 0.5242 - loss: 0.5242
Epoch 6/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 24s 24ms/step - classifier/loss: 0.5304 - loss: 0.5304
Epoch 7/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 25s 25ms/step - classifier/loss: 0.5274 - loss: 0.5274
Epoch 8/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 24s 24ms/step - classifier/loss: 0.5285 - loss: 0.5285
Epoch 9/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 22s 22ms/step - classifier/loss: 0.5171 - loss: 0.5171
Epoch 10/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 24s 24ms/step - classifier/loss: 0.5305 - loss: 0.5305
